## Save & Load

In case of `cupy` we could transform to `numpy` so we will only consider saving as `numpy`

In [1]:
import numpy as np

x = np.array([1, 2, 3])
np.save('test.npy', x)

x = np.load('test.npy')
print(x)

[1 2 3]


**How about saving several arrays?**

In [2]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])

np.savez('test.npz', x1=x1, x2=x2)

arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)

[1 2 3]
[4 5 6]


**`dict` use - Above is the same as below**

In [3]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
data = {'x1': x1, 'x2': x2}

np.savez('test.npz', **data)

arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)

[1 2 3]
[4 5 6]


### `np.savez_compressed` = `np.savez` + compress

## `Flatten` Layer's `parameters`

In [1]:
from dezero import Parameter
from dezero import Layer

layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

```python
class Layer:
    ...
    def _flatten_params(self, params_dict, parent_key=""):
        for name in self._params:
            obj = self.__dict__[name]
            key = parent_key + '/' + name if parent_key else name

            if isinstance(obj, Layer):
                obj._flatten_params(params_dict, key)
            else:
                params_dict[key] = obj
```

In [2]:
params_dict = {}
layer._flatten_params(params_dict)
print(params_dict)

{'p3': Variable(3), 'l1/p1': Variable(1), 'p2': Variable(2)}


## Layer class `save` & `load`

```python
import os

class Layer:
    ...
    def save_weights(self, path):
        # was model using gpu before save?
        using_gpu = False

        if self.gpu:
            self.to_cpu()
            using_gpu = True

        params_dict = {}
        self._flatten_params(params_dict)
        array_dict = {key: param.data for key, param in params_dict.items()}

        try:
            np.savez_compressed(path, **array_dict)
        except (Exception, KeyboardInterrupt) as e:
            if os.path.exists(path):
                os.remove(path)
            raise

        # if model was using gpu restore it!
        if using_gpu:
            self.to_gpu()

    def load_weights(self, path):
        npz = np.load(path)
        params_dict = {}
        self._flatten_params(params_dict)
        for key, param in params_dict.items():
            param.data = npz[key]
```

## Load & Save test with `MNIST`

In [1]:
import time
import math
import numpy as np

import dezero
from dezero import optimizers
from dezero import DataLoader

import dezero.functions as F
from dezero.models import MLP
from dezero.datasets import MNIST

In [4]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

### with `gpu`

In [6]:
import os

if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()

for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    elasped_time = time.time() - start
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))

model.save_weights('my_mlp.npz')

epoch : 1
train loss: 0.3995, accuracy: 0.8851, time: 4.5996[sec]
epoch : 2
train loss: 0.3504, accuracy: 0.8976, time: 4.4648[sec]
epoch : 3
train loss: 0.3289, accuracy: 0.9047, time: 4.5244[sec]
epoch : 4
train loss: 0.3160, accuracy: 0.9085, time: 4.5520[sec]
epoch : 5
train loss: 0.3058, accuracy: 0.9111, time: 4.5762[sec]


### Reset and test :)

In [7]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

In [8]:
import os

if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')

if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()
    
for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    elasped_time = time.time() - start
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))

epoch : 1
train loss: 0.2977, accuracy: 0.9142, time: 4.5690[sec]
epoch : 2
train loss: 0.2922, accuracy: 0.9158, time: 4.4997[sec]
epoch : 3
train loss: 0.2861, accuracy: 0.9175, time: 4.6459[sec]
epoch : 4
train loss: 0.2818, accuracy: 0.9192, time: 4.9926[sec]
epoch : 5
train loss: 0.2768, accuracy: 0.9208, time: 5.0784[sec]


### Working GOOD!

### with `cpu`

In [5]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

In [6]:
for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    elasped_time = time.time() - start
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))

model.save_weights('my_mlp.npz')

epoch : 1
train loss: 0.9763, accuracy: 0.7235, time: 5.8883[sec]
epoch : 2
train loss: 0.4001, accuracy: 0.8842, time: 5.6606[sec]
epoch : 3
train loss: 0.3522, accuracy: 0.8978, time: 5.9509[sec]
epoch : 4
train loss: 0.3294, accuracy: 0.9044, time: 5.5500[sec]
epoch : 5
train loss: 0.3151, accuracy: 0.9078, time: 6.0811[sec]


### Reset and test :)

In [10]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

In [11]:
import os

if os.path.exists('my_mlp.npz'):
    model.load_weights('my_mlp.npz')
    
for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    elasped_time = time.time() - start
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))

epoch : 1
train loss: 0.3067, accuracy: 0.9108, time: 5.6444[sec]
epoch : 2
train loss: 0.2988, accuracy: 0.9131, time: 5.5396[sec]
epoch : 3
train loss: 0.2925, accuracy: 0.9165, time: 5.7961[sec]
epoch : 4
train loss: 0.2868, accuracy: 0.9175, time: 6.0873[sec]
epoch : 5
train loss: 0.2810, accuracy: 0.9197, time: 6.2140[sec]


### Also working GOOD!

### Save while training `gpu`

In [4]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

In [5]:
if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    model.to_gpu()
    
for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    elasped_time = time.time() - start
    
    print(f'model saved at epoch: {epoch}')
    model.save_weights('my_mlp.npz')
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))

model saved at epoch: 0
epoch : 1
train loss: 0.9520, accuracy: 0.7252, time: 5.4288[sec]
model saved at epoch: 1
epoch : 2
train loss: 0.4002, accuracy: 0.8845, time: 4.9452[sec]
model saved at epoch: 2
epoch : 3
train loss: 0.3516, accuracy: 0.8979, time: 4.9315[sec]
model saved at epoch: 3
epoch : 4
train loss: 0.3300, accuracy: 0.9038, time: 4.9317[sec]
model saved at epoch: 4
epoch : 5
train loss: 0.3160, accuracy: 0.9077, time: 4.9161[sec]


In [6]:
model.gpu

True

### we won't test loading because it is already proved working above

### Save while training `cpu`

In [7]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
train_loader = DataLoader(train_set, batch_size)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

In [8]:
for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)
    elasped_time = time.time() - start
    
    print(f'model saved at epoch: {epoch}')
    model.save_weights('my_mlp.npz')
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))

model saved at epoch: 0
epoch : 1
train loss: 0.9519, accuracy: 0.7238, time: 5.9801[sec]
model saved at epoch: 1
epoch : 2
train loss: 0.4017, accuracy: 0.8851, time: 7.4085[sec]
model saved at epoch: 2
epoch : 3
train loss: 0.3517, accuracy: 0.8974, time: 7.1948[sec]
model saved at epoch: 3
epoch : 4
train loss: 0.3296, accuracy: 0.9035, time: 7.2806[sec]
model saved at epoch: 4
epoch : 5
train loss: 0.3162, accuracy: 0.9083, time: 7.1214[sec]


In [9]:
model.gpu

False

### we won't test loading because it is already proved working above